In [ ]:
import cv2
import numpy as np
import torch
import os
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
from IPython.display import display, clear_output
from ipywidgets import widgets, interact
from libreface.core.inference_engine import InferenceEngine
from libreface.modules import detection, landmarking, landmarks_postprocessing, action_units

ModuleNotFoundError: No module named 'libreface.core'

: 

In [ ]:
def setup_libreface_engine():
    # Set device (GPU if available, otherwise CPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    # Initialize inference engine
    engine = InferenceEngine(device=device)
    
    # Add required modules for AU detection
    engine.add_module("face_detector", detection.RetinaFace())  # Face detection
    engine.add_module("landmark_detector", landmarking.Fan2D())  # Facial landmark detection
    engine.add_module("landmark_postprocessor", landmarks_postprocessing.FANPostProcessor())  # Refine landmarks
    engine.add_module("au_detector", action_units.AU())  # Action unit detection
    
    return engine

In [ ]:
def process_video_for_au(video_path, output_dir=None, display_in_notebook=True, max_frames=None):
    """
    Process a video to detect action units using LibreFace
    
    Args:
        video_path: Path to the video file
        output_dir: Directory to save results (if None, results won't be saved)
        display_in_notebook: Whether to display frames in the notebook
        max_frames: Maximum number of frames to process (None for all frames)
        
    Returns:
        Dictionary with frame indices as keys and AU predictions as values
    """
    # Create output directory if specified
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)
    
    # Initialize the video capture
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError(f"Could not open video file: {video_path}")
    
    print(f"Successfully opened video: {video_path}")
    
    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    print(f"Video properties: {frame_width}x{frame_height}, {fps} FPS, {total_frames} frames total")
    
    # Set up LibreFace inference engine
    engine = setup_libreface_engine()
    
    # Initialize video writer if output_dir is specified
    out = None
    if output_dir:
        output_path = os.path.join(output_dir, f"{Path(video_path).stem}_au_output.mp4")
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))
        print(f"Output video will be saved to: {output_path}")
    
    # Dictionary to store AU results
    au_results = {}
    
    # For notebook display
    plt.figure(figsize=(12, 10))
    plt.axis('off')
    display_handle = display(None, display_id=True)
    
    frame_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret or (max_frames is not None and frame_idx >= max_frames):
            break
        
        # BGR to RGB conversion
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Process the frame with LibreFace
        result = engine.process({"image": rgb_frame})
        
        # Extract AU predictions
        if "aus" in result and result["aus"] is not None and len(result["aus"]) > 0:
            au_results[frame_idx] = result["aus"]
        
        # Visualize results on the frame
        vis_frame = frame.copy()
        
        # Draw face bounding boxes
        if "detections" in result and result["detections"] is not None:
            for det in result["detections"]:
                box = det.box.astype(int)
                cv2.rectangle(vis_frame, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)
        
        # Draw landmarks
        if "landmarks" in result and result["landmarks"] is not None:
            for lmks in result["landmarks"]:
                for (x, y) in lmks.astype(int):
                    cv2.circle(vis_frame, (x, y), 2, (0, 0, 255), -1)
        
        # Display AU values
        if frame_idx in au_results:
            aus = au_results[frame_idx]
            for i, (face_aus, face_scores) in enumerate(aus):
                y_offset = 30
                for j, (au, score) in enumerate(zip(face_aus, face_scores)):
                    text = f"AU{au}: {score:.2f}"
                    cv2.putText(vis_frame, text, (10, y_offset + j*20), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
        
        # Write frame to output video
        if out:
            out.write(vis_frame)
        
        # Display the frame in the notebook
        if display_in_notebook and frame_idx % 5 == 0:  # Update display every 5 frames for efficiency
            rgb_vis_frame = cv2.cvtColor(vis_frame, cv2.COLOR_BGR2RGB)
            plt.imshow(rgb_vis_frame)
            plt.title(f"Frame {frame_idx}/{total_frames}")
            display_handle.update(plt.gcf())
            plt.clf()
            plt.axis('off')
        
        frame_idx += 1
        
        # Print progress every 100 frames
        if frame_idx % 100 == 0:
            print(f"Processed {frame_idx} frames out of {total_frames if max_frames is None else max_frames}")
    
    # Cleanup
    cap.release()
    if out:
        out.release()
    plt.close()
    
    print(f"Processed {frame_idx} frames total")
    print(f"Detected faces with AU in {len(au_results)} frames")
    
    return au_results

In [ ]:
def save_au_results_to_csv(au_results, output_path):
    """
    Save AU results to a CSV file
    
    Args:
        au_results: Dictionary with frame indices as keys and AU predictions as values
        output_path: Path to save the CSV file
    """
    data = []
    for frame_idx, aus in au_results.items():
        for face_idx, (face_aus, face_scores) in enumerate(aus):
            for au, score in zip(face_aus, face_scores):
                data.append({
                    'frame_idx': frame_idx,
                    'face_idx': face_idx,
                    'au': f'AU{au}',
                    'score': score
                })
    
    df = pd.DataFrame(data)
    df.to_csv(output_path, index=False)
    print(f"Saved AU results to {output_path}")
    return df

In [ ]:
def plot_au_data(au_results):
    """
    Plot AU scores over time for visualization
    """
    # Convert to dataframe
    data = []
    for frame_idx, aus in au_results.items():
        for face_idx, (face_aus, face_scores) in enumerate(aus):
            for au, score in zip(face_aus, face_scores):
                data.append({
                    'frame_idx': frame_idx,
                    'face_idx': face_idx,
                    'au': f'AU{au}',
                    'score': score
                })
    
    df = pd.DataFrame(data)
    if len(df) == 0:
        print("No AU data to plot")
        return
    
    # Get unique AUs and faces
    unique_aus = sorted(df['au'].unique())
    unique_faces = sorted(df['face_idx'].unique())
    
    # Create one plot per face
    for face_idx in unique_faces:
        face_data = df[df['face_idx'] == face_idx]
        plt.figure(figsize=(15, 8))
        
        for au in unique_aus:
            au_data = face_data[face_data['au'] == au]
            if not au_data.empty:
                plt.plot(au_data['frame_idx'], au_data['score'], label=au)
        
        plt.title(f"Action Unit Scores for Face #{face_idx}")
        plt.xlabel("Frame")
        plt.ylabel("AU Score")
        plt.grid(True, alpha=0.3)
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.tight_layout()
        plt.show()

In [ ]:
# Set the path to your video file
video_path = "path/to/your/video.mp4"  # Change this to your video path

# Set output directory (optional)
output_dir = "results"  # Change or set to None if you don't want to save results

# Maximum frames to process (optional, set to None to process all frames)
max_frames = 300  # Process first 300 frames, set to None for full video

In [ ]:
# Run the AU detection
au_results = process_video_for_au(
    video_path=video_path,
    output_dir=output_dir,
    display_in_notebook=True,
    max_frames=max_frames
)

In [ ]:
if output_dir and au_results:
    output_csv = os.path.join(output_dir, f"{Path(video_path).stem}_au_results.csv")
    df = save_au_results_to_csv(au_results, output_csv)
    
    # Display the first few rows
    display(df.head())

In [ ]:
# Plot the AU scores over time
if au_results:
    plot_au_data(au_results)